#Investigating The Effect of Hurricane Maria On Smallholder Farms In Puerto Rico 
(Santhi Davedu, Inigo Peng, Vineeth Jason Putti, Yunfan Zhou)

##Background
2017’s Hurricane Maria struck the United States unincorporated territory of Puerto Rico as a category 4 hurricane, causing 2,975 deaths. The archipelago’s agriculture, which was experiencing production improvement during Puerto Rico’s financial crisis, was decimated. Damages to production were estimated to reach approximately 200 million dollars, and 80% of its crop value was devastated. Hurricanes in the Atlantic are likely to become more intense and persistent in the upcoming years due to climate change, which will have profound impacts in island countries and territories, especially in their agricultural systems and food security. Thus, understanding how smallholder farmers are affected by hurricanes is critical for their future adaptation, like what strategies they may use to increase their resiliency in the future.

##Problem Statement
We aim to conduct landcover change anaylsis through measurement of measurement of normalized vegetation index. We are interested to see if there is a significant change in landcover change before and after Hurricane Maria. This is part of a larger investigation in combining different data to identify what tools, inputs and management strategies could most effectively contribute to an increase in the resiliance of a given farming operation. 

##Methodology
This is the second part of the project, where moderate spatial resolution satellite image taken by Sentinel-2 of European Space Agency (ES) is used to analyze smallholder farm data provided by the Environmental Spatial Analysis (ESA) Lab at SEAS. This part of the project is conducted through Google Earth Engine (GEE) Python API. We first conducted some exploratory mapping by creating tri-month false colour composits of Sentinel 2 Level 2A images from June of 2017 to December of 2019. We then proceeded to visualize the NDVI for the composites. To investigate the landcover changes of the individual farms, we first prepared the farm data. The excel coordinates was converted into json file for easy access of geometry (point, coordinates) and properties (farm number, owner of the farm). We overlayed the farm points onto the map and created 1km buffers around all of them. Lastly, we parsed the json file and calculated the NDVI mean and standard deviation of farm. 
##Python Script

In [26]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import csv

In [2]:
# Add Google Map
Map = emap.Map(center=[18,-66], zoom=9)
Map.addLayerControl()
Map

Map(center=[18, -66], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [3]:
#Import farm json file
file_path = 'PR_farms.json'
with open(file_path) as f:
    json_data = json.load(f)


In [4]:
# Exploratory work with the farm data 
farms = json_data
properties = farms['features'][0]['properties']
geometry = farms['features'][0]['geometry']
farm_data = farms['features']
#Convert json data into Google Earth Engine Features
farm_data_features_list = [ee.Feature(farms['features'][i]['geometry'],farms['features'][i]['properties']) for i in range(len(farm_data))]


In [5]:
#Append farm points and buffer onto the map
for item in farm_data_features_list:
    Map.addLayer(item, {}, 'Small Holder Farms Research Location Feature')

for item in farm_data_features_list:
    new_geometry = item.buffer(1000)
    Map.addLayer(new_geometry, {'color': 'red'},'buffer')
    

In [6]:
# cloud mask function
def maskS2clouds(image):
#     bits 10 and 11 are cloud and cirrus 
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
#     both needs to be set to zero - clear condition
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))
#     return the masked and scaled data without the QA bands
    return image.updateMask(mask).divide(10000)\
        .select("B.*")\
        .copyProperties(image, ['system:time_start'])

In [7]:
#calculating NDVI
def addNDVI(img):
  ndvi = img.normalizedDifference(['B8', 'B4']).rename('ndvi')
  return img.addBands([ndvi])

In [8]:
# Setting query boundary
PR_polygon = ee.Geometry.Polygon([[[-66.97540283203125, 18.04403274297545],\
                                 [-65.93170166015625, 18.04403274297545],\
                                 [-65.93170166015625, 18.41447273166262],\
                                 [-66.97540283203125, 18.41447273166262],\
                                 [-66.97540283203125, 18.04403274297545]]])

In [9]:
#Creating composites - composite between March and June had missing pixels due to cloud mask
Start0 = ee.Date('2017-03-28')
End0 = ee.Date('2017-6-30')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start0, End0)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite0 = img.median()

In [100]:
Map.addLayer(composite0, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite April - June 2017')

In [10]:
Start1 = ee.Date('2017-07-01')
End1 = ee.Date('2017-09-30')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start1, End1)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite1 = img.median()

In [102]:
Map.addLayer(composite1, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite July - Septeber 2017')

In [11]:
Start2 = ee.Date('2017-10-01')
End2 = ee.Date('2018-12-31')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start2, End2)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite2 = img.median()

In [ ]:
Map.addLayer(composite2, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite Oct - Dec 2017')

In [12]:
Start3 = ee.Date('2018-01-01')
End3 = ee.Date('2018-03-31')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start3, End3)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite3 = img.median()

In [ ]:
Map.addLayer(composite3, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite Jan - Mar 2018')

In [13]:
Start4 = ee.Date('2018-04-01')
End4 = ee.Date('2018-06-30')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start4, End4)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite4 = img.median()

In [ ]:
Map.addLayer(composite4, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite April - June 2018')

In [14]:
Start5 = ee.Date('2018-07-01')
End5 = ee.Date('2018-09-30')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start5, End5)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite5 = img.median()

In [ ]:
Map.addLayer(composite5, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite July - Septemer 2018')

In [15]:
Start6 = ee.Date('2018-10-01')
End6 = ee.Date('2018-12-31')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start6, End6)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite6 = img.median()

In [ ]:
Map.addLayer(composite6, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite October - December 2018')

In [16]:
Start7 = ee.Date('2019-1-01')
End7 = ee.Date('2019-3-31')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start7, End7)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite7 = img.median()

In [ ]:
Map.addLayer(composite7, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite January - March 2019')

In [17]:
Start8 = ee.Date('2019-4-01')
End8 = ee.Date('2019-6-30')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start8, End8)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite8 = img.median()

In [ ]:
Map.addLayer(composite8, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite March- June 2019')

In [18]:
Start9 = ee.Date('2019-07-01')
End9 = ee.Date('2019-09-30')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start9, End9)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite9 = img.median()

In [ ]:
Map.addLayer(composite9, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite July - September 2019')

In [19]:
Start10 = ee.Date('2019-10-01')
End10 = ee.Date('2019-12-31')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start10, End10)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite10 = img.median()

In [ ]:
Map.addLayer(composite10, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR Composite October - December 2019')

In [20]:
composite_list = [composite0, composite1, composite2, composite3, composite4, composite5, composite6, composite7, composite8, composite9,
                 composite10]

In [21]:
#Single image ndvi calculation
composite1_ndvi = addNDVI(composite1)
ndvi1 = composite1_ndvi.select(['ndvi'])

In [22]:
# Exploratory NDVI visualization
# Green = High NDVI value, Yellow = Low NDVI value
visParams_ndvi = {'bands': ['ndvi'], 'min': -0.2, 'max': 0.8, 'palette': 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400,' + \
    '3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'}
Map.addLayer(ndvi1,visParams_ndvi,'Sentinel-2 NDVI Oct - Dec 2019(sample image)')

In [23]:
# Adding NDVI bands to all composites
ndvi_dict = {}
for i in range(len(composite_list)):
    ndvi_dict['ndvi %s' %(i)] = composite_list[i]
for k,v in ndvi_dict.items():
    ndvi_dict[k] = addNDVI(v).select(['ndvi'])

In [24]:
#Calculation for individual farm NDVI mean and SD
#Code showns only the NDVI mean from the first composite (March - June of 2017)
new_geometry_dict ={}
for i in range(len(farm_data)):
    new_geometry_dict['farm %s'%(i)] = ee.Geometry.Point(farms['features'][i]['geometry']['coordinates']).buffer(1000)
    
reducers = ee.Reducer.mean().combine(**{
  'reducer2': ee.Reducer.stdDev(),
  'sharedInputs': True
})

# Use the combined reducer to get the mean and SD of the image.
# We could see the individal farm NDVI has been calculated
# Other sentinel2  Peurto Rico Project Images could be uploaded onto Google Earth Engine and we could try calculating the NDVI value the farms in those images. 
stats_dict ={}
for i in range(len(farm_data)):
    for n in range(len(composite_list)):
        stats_dict['farm %s, composite %s' %(i, n)] = ndvi_dict['ndvi %s'%(n)].reduceRegion(**{
        'reducer': reducers,
        'bestEffort': True,
        'scale': 30,
        'geometry': new_geometry_dict['farm %s'%(i)]
})
for farm in stats_dict.keys():
    print (farm)
    print (stats_dict[farm].getInfo())
                                    

farm 0, composite 0
{'ndvi_mean': 0.6332507061786319, 'ndvi_stdDev': 0.1739024063953939}
farm 0, composite 1
{'ndvi_mean': 0.7165135398897369, 'ndvi_stdDev': 0.07219930376415218}
farm 0, composite 2
{'ndvi_mean': 0.7061207345285844, 'ndvi_stdDev': 0.08638071503480232}
farm 0, composite 3
{'ndvi_mean': 0.7140707999320582, 'ndvi_stdDev': 0.09180558884781748}
farm 0, composite 4
{'ndvi_mean': 0.6538867289390089, 'ndvi_stdDev': 0.10589986302941512}
farm 0, composite 5
{'ndvi_mean': 0.7005527891924914, 'ndvi_stdDev': 0.08898002282654198}
farm 0, composite 6
{'ndvi_mean': 0.7321340374920733, 'ndvi_stdDev': 0.09690954102155408}
farm 0, composite 7
{'ndvi_mean': 0.6913389697796902, 'ndvi_stdDev': 0.1055597940932304}
farm 0, composite 8
{'ndvi_mean': 0.6977948366848092, 'ndvi_stdDev': 0.10233561979674659}
farm 0, composite 9
{'ndvi_mean': 0.7230934880356888, 'ndvi_stdDev': 0.09510156893935746}
farm 0, composite 10
{'ndvi_mean': 0.7297479548400456, 'ndvi_stdDev': 0.09479368530918128}
farm 1, com

{'ndvi_mean': 0.7033221045983815, 'ndvi_stdDev': 0.061445630543373125}
farm 8, composite 6
{'ndvi_mean': 0.7589753392837264, 'ndvi_stdDev': 0.07428681605713544}
farm 8, composite 7
{'ndvi_mean': 0.7044082035890231, 'ndvi_stdDev': 0.10597834444152719}
farm 8, composite 8
{'ndvi_mean': 0.7386336997602744, 'ndvi_stdDev': 0.09182127726275272}
farm 8, composite 9
{'ndvi_mean': 0.7290662736336875, 'ndvi_stdDev': 0.06358868950356528}
farm 8, composite 10
{'ndvi_mean': 0.7481293013475342, 'ndvi_stdDev': 0.07420333419799319}
farm 9, composite 0
{'ndvi_mean': 0.492859529556065, 'ndvi_stdDev': 0.1248835873366531}
farm 9, composite 1
{'ndvi_mean': 0.6994091284509374, 'ndvi_stdDev': 0.10064713430231888}
farm 9, composite 2
{'ndvi_mean': 0.7176068314144126, 'ndvi_stdDev': 0.06138061806303244}
farm 9, composite 3
{'ndvi_mean': 0.7115519065472053, 'ndvi_stdDev': 0.07508165979697248}
farm 9, composite 4
{'ndvi_mean': 0.6082014263799155, 'ndvi_stdDev': 0.16178133326773958}
farm 9, composite 5
{'ndvi_mea

{'ndvi_mean': 0.6555110027398646, 'ndvi_stdDev': 0.08446639717986765}
farm 16, composite 9
{'ndvi_mean': 0.6382254492876952, 'ndvi_stdDev': 0.08418085714810054}
farm 16, composite 10
{'ndvi_mean': 0.7007297397561706, 'ndvi_stdDev': 0.0735799248726129}
farm 17, composite 0
{'ndvi_mean': 0.5643188199500585, 'ndvi_stdDev': 0.20744845542257806}
farm 17, composite 1
{'ndvi_mean': 0.6632876424823413, 'ndvi_stdDev': 0.1084540147616758}
farm 17, composite 2
{'ndvi_mean': 0.6375845117695742, 'ndvi_stdDev': 0.10533135408081931}
farm 17, composite 3
{'ndvi_mean': 0.6498252042813905, 'ndvi_stdDev': 0.12264454970813957}
farm 17, composite 4
{'ndvi_mean': 0.622063443542215, 'ndvi_stdDev': 0.11267895125485096}
farm 17, composite 5
{'ndvi_mean': 0.6032211662977361, 'ndvi_stdDev': 0.10920690819092138}
farm 17, composite 6
{'ndvi_mean': 0.6654287171133789, 'ndvi_stdDev': 0.11771575348759143}
farm 17, composite 7
{'ndvi_mean': 0.633072599855569, 'ndvi_stdDev': 0.11546221159231947}
farm 17, composite 8
{'

In [27]:
with open('dict.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in stats_dict.items():
       writer.writerow([key, value.getInfo()])

In [32]:
#This was an attempt to export the images from Google Earth Engine
#I have tried many ways such as save to drive or save to computer but they all show up corrupted.
#The code seems correct - it might be a python API problem. 
task_config = {
    'folder': 'gee', # output Google Drive folder
    'region': PR_polygon,     # roi 
    'scale': 100,       # image resolution
    'crs': 'EPSG:4326',
    'maxPixels': 1.0E13,
    'fileFormat': 'GeoTIFF'
    }
image = composite9
description = 'Composite 9'

# Export image to Google Drive
task = ee.batch.Export.image.toDrive(image, description, **task_config)
print(task)
task.start()
print("Exporting {}".format(description))

<Task EXPORT_IMAGE: Composite 9 (UNSUBMITTED)>
Exporting Composite 9


#Future Work
The next step is to create an organized chart to consolidate all of the ndvi values for each farm. These data be plotted to show how the NDVI values have changed from 2017 until now. The composite images also needs to be properly exported. Right now, the exported images shows up corrupted and are unusable. More work needs to be done to either investigate what is wrong with  image export in Google Earth Engine Python API or the python script could be converted to Javascript and we could run it through the Javascript API. We could also put togeter as a timelapse for visualization. Further comparison with data before 2017 could give us a better understanding of the changes in the landcover. These data could then be used as reference points for further data fusion work.